In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import torchvision
import torch
import torch.nn as nn

# dom_click_table = pd.read_csv('./DomClickData/Dom_click_data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Diplom/data_hse.zip

unzip:  cannot find or open /content/drive/MyDrive/Diplom/data_hse.zip, /content/drive/MyDrive/Diplom/data_hse.zip.zip or /content/drive/MyDrive/Diplom/data_hse.zip.ZIP.


In [ ]:
dom_click_table = pd.read_csv('/content/drive/MyDrive/Diplom/after_preproc.csv')

In [ ]:
dom_click_table.head()

In [ ]:
dom_click_table['im_name'] = ''
n = len(dom_click_table)
for i in range(n):
    dom_click_table['im_name'].iloc[i] = dom_click_table['first_value'][i].split('/')[-1]

<ipython-input-6-503f6149720e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dom_click_table['im_name'].iloc[i] = dom_click_table['first_value'][i].split('/')[-1]


In [ ]:
dom_click_table[dom_click_table['shows'] == 0]

In [ ]:
prob_table = dom_click_table.groupby(by=["im_name"]).sum().reset_index()[['im_name', 'clicks', 'shows', 'price']]

<ipython-input-89-0a6392c0ac8b>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  prob_table = dom_click_table.groupby(by=["im_name"]).sum().reset_index()[['im_name', 'clicks', 'shows', 'price']]


In [ ]:
# import 
prob_table.groupby(by=['clicks']).count()

,im_name,shows,price
clicks,,,
0,1391,1391,1391
1,219,219,219
2,74,74,74
3,25,25,25
4,15,15,15
5,12,12,12
6,8,8,8
7,6,6,6
8,5,5,5


In [ ]:
prob_table['prob'] = None
prob_table['prob'].loc[prob_table['shows'] != 0] = prob_table['clicks'] / prob_table['shows']
prob_table['prob'].loc[prob_table['shows'] == 0] = 0
prob_table.sample(5)

<ipython-input-91-61adaa35f993>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_table['prob'].loc[prob_table['shows'] != 0] = prob_table['clicks'] / prob_table['shows']
<ipython-input-91-61adaa35f993>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_table['prob'].loc[prob_table['shows'] == 0] = 0


,im_name,clicks,shows,price,prob
462,43db52820c909b5cc6a87f1d31a29fa55bd08aab.jpg,0,3,320000.0,0.0
1128,a7a95a1621d91286e737bad984d0b8b8756864ff.jpg,0,10,120000.0,0.0
502,4a45d88ccae341c9b0a67004c501bebe.jpg,0,16,294000.0,0.0
775,7291f662c94737643698b1b652fd38466ea15817.jpg,0,3,3600000.0,0.0
608,593a5e2443214b5e80eaa69546a379de.jpg,3,73,216000.0,0.041096


In [ ]:
prob_table.shape

(1765, 5)

In [ ]:
prob_table = prob_table.drop(prob_table['prob'].loc[prob_table['shows'] == 0].index)

In [ ]:
prob_table = prob_table.reset_index()

In [ ]:
class Dom_Click_Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None, transform=None):
        self.const_dir = './DomClickData/dom_click_data/data_hse/'
        self.transform = transform
        self.num_elem = len(X)
        self.X = X
        self.target = y

    def __getitem__(self, idx):
        im_name = self.X.iloc[idx]
        image = torchvision.io.read_image(im_name) / 255

        if (self.transform is not None):
            image = self.transform(image)
        if (self.target is not None):
            # return image, torch.Tensor(self.X.iloc[idx][['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']])
            return image, self.target.iloc[idx]
        return image
    
    def __len__(self):
        return self.num_elem

In [ ]:
import torchvision.transforms as T

data_transform = nn.Sequential(
    T.CenterCrop(240),
    T.Resize(256),
    T.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
)

In [ ]:
prob_table.shape

(1736, 5)

In [ ]:
# dom_click_table[dom_click_table['im_name'] == '3e88729975ca610261e321a51a7b83932ef50565.jpg']
# dom_click_table[dom_click_table['im_name'] == 'fdf2eac88c8b06eda29c03ee55ee4dc2d94ddeed.jpg']
# dom_click_table[dom_click_table['im_name'] == '8d82c8b415cbd34a2c5606e2c881609cd10eaed2.jpg']


In [ ]:
# dom_click_table = dom_click_table.drop([3733, 4294, 4295, 4296])

In [ ]:
prob_table['im_name'] = 'data_hse/' + prob_table['im_name']

In [ ]:
from torch.utils.data import DataLoader
dataset = Dom_Click_Dataset(prob_table['im_name'], None, transform=data_transform)
batch_size=1
dataset_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
drop_list = []
for i in range(len(prob_table)):
    try:
        dataset[i].shape
    except:
        drop_list += [i]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
prob_table = prob_table.drop(drop_list)

In [ ]:
prob_table = prob_table.reset_index().drop(columns = ['level_0', 'index'])

In [ ]:
dataset = Dom_Click_Dataset(prob_table['im_name'], None, transform=data_transform)
batch_size=1
dataset_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
prob_table['im_name'][1]

'data_hse/0025305cb40c5bae7d1acba5e079eb7e8c02a8a8.jpg'

In [ ]:
torchvision.io.read_image(prob_table['im_name'][0])

tensor([[[104, 104, 106,  ..., 117, 117, 116],
         [106, 106, 106,  ..., 118, 118, 117],
         [105, 105, 106,  ..., 119, 119, 119],
         ...,
         [ 90,  88,  89,  ...,  16,  16,  16],
         [ 93,  91,  93,  ...,  18,  21,  20],
         [ 95,  92,  93,  ...,  20,  23,  21]],

        [[ 79,  79,  81,  ..., 110, 110, 109],
         [ 81,  81,  81,  ..., 111, 111, 110],
         [ 80,  80,  81,  ..., 112, 112, 112],
         ...,
         [ 67,  65,  66,  ...,  11,  14,  14],
         [ 70,  68,  70,  ...,  13,  16,  15],
         [ 72,  69,  70,  ...,  15,  18,  16]],

        [[ 57,  57,  59,  ...,  92,  94,  93],
         [ 59,  59,  59,  ...,  93,  95,  94],
         [ 58,  58,  59,  ...,  96,  96,  96],
         ...,
         [ 53,  51,  52,  ...,  17,  19,  19],
         [ 56,  54,  56,  ...,  19,  22,  21],
         [ 58,  55,  56,  ...,  21,  24,  22]]], dtype=torch.uint8)

In [ ]:
model = torch.load('/content/drive/MyDrive/Diplom/eff_net_model_86_48')
model.eval()

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x

In [ ]:
model.classifier = Identity()

In [ ]:
prob_table['img_embed'] = None

In [ ]:
from tqdm.notebook import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

predictions = np.array([])
true_labels = np.array([])
i = 0
for x_batch in tqdm(dataset_loader):
    try:
        data = x_batch.to(device)
        # target = y_batch.cpu().numpy()
        output = model(data)
        prob_table['img_embed'].iloc[i] = output.cpu().detach().numpy()
    except:
        print(i)
    i += 1

  0%|          | 0/1547 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
<ipython-input-109-8718708e462e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_table['img

In [ ]:
prob_table

,im_name,clicks,shows,price,prob,img_embed
0,data_hse/0022feeaedac4b959c5c6e375632316d.jpg,1,13,160000.0,0.076923,"[[0.052418582, -0.12397501, 0.17204553, 0.4417..."
1,data_hse/0025305cb40c5bae7d1acba5e079eb7e8c02a...,0,1,245000.0,0.0,"[[-0.012044404, -0.10706582, 0.9206433, -0.079..."
2,data_hse/011e50153f2ec1796b12e1e8b0cf1d1089a74...,0,2,165000.0,0.0,"[[0.06785435, -0.076649815, 0.66985345, 0.7305..."
3,data_hse/0132a35df19e456991f7dcc8db13e6e8.jpg,0,4,156000.0,0.0,"[[-0.04730807, -0.15744582, 0.0014863298, -0.0..."
4,data_hse/0146c6f8ab17ae16dbfcd4bf07192bf9346af...,0,3,940000.0,0.0,"[[0.050134186, 0.7944262, 0.39178973, -0.08922..."
...,...,...,...,...,...,...
1542,data_hse/ff07cbbb93b1a1dcafee2fd86b59ea4acf847...,2,11,129000.0,0.181818,"[[0.034675475, -0.059836615, 0.22403479, -0.08..."
1543,data_hse/ff51e955da55c3c463f0607a9ec85268b77aa...,0,59,56000.0,0.0,"[[-0.08442485, -0.09079578, -0.11027831, 0.842..."
1544,data_hse/ffb036fad456a46d6bdad135254c2abbc906d...,0,2,260000.0,0.0,"[[0.027945686, 0.48572183, 0.31887877, -0.0540..."
1545,data_hse/ffe57b10b6d9a064e83c5bb958a4d024f2705...,0,4,190000.0,0.0,"[[-0.07170361, -0.1269826, -0.13211381, 0.1279..."


In [ ]:
df3 = pd.DataFrame()

In [ ]:
prob_table.shape
len(prob_table)

1547

In [ ]:
for i in range(len(prob_table)):
    df3 = pd.concat([df3, pd.DataFrame(prob_table['img_embed'].iloc[i].tolist())])

In [ ]:
df3 = df3.reset_index().drop(columns = ['index'])

In [ ]:
df3['clicks'] = prob_table['clicks']
df3['shows'] = prob_table['shows']
df3['price'] = prob_table['price']
df3['prob'] = prob_table['prob']
df3['im_name'] = prob_table['im_name']

In [ ]:
df3.to_csv('Domclick_embeds_after_rep_type_classification')

# Clustering

In [ ]:
from sklearn.cluster import KMeans
num = 7
kmeans = KMeans(n_clusters=num)
kmeans.fit_predict(df3.iloc[:, range(0, 1280)])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([1, 2, 0, ..., 6, 3, 6], dtype=int32)

In [ ]:
labels = kmeans.labels_

In [ ]:
for i in range(num):
    print('NUM ELEMS IN CLASS:', (labels == i).sum(), ' MEAN:', prob_table[labels == i]['prob'].mean())

NUM ELEMS IN CLASS: 198  MEAN: 0.006705019590854686
NUM ELEMS IN CLASS: 136  MEAN: 0.023176402986770043
NUM ELEMS IN CLASS: 253  MEAN: 0.02314993425226052
NUM ELEMS IN CLASS: 294  MEAN: 0.02538638361353595
NUM ELEMS IN CLASS: 98  MEAN: 0.031118589335116582
NUM ELEMS IN CLASS: 260  MEAN: 0.007442814572958585
NUM ELEMS IN CLASS: 308  MEAN: 0.022327317809190037


In [ ]:
prob_table.sample(5)#['prob'].mean()

,im_name,clicks,shows,prob,img_embed
1092,5dee3b904d3950c75c86e8d6e05abd3e854a4f0e.jpg,0,1,0.000000,"[-0.113018826, -0.14722648, -0.15318014, -0.11..."
442,25dfd7e8a188827413d1af75d5e7b375d0746267.jpg,4,39,0.102564,"[-0.12542851, -0.047838893, -0.12912802, -0.15..."
2193,bf9470098ff44235b26dd0b76ac11f93.jpg,1,33,0.030303,"[0.09229316, -0.19861597, -0.12639543, -0.1510..."
843,49195d2a1271d5b3086558c30cf585e136b36474.jpg,0,38,0.000000,"[-0.10827659, -0.15090899, -0.15194692, 0.3595..."
2273,c627045cd0e12c4f26ede9b7f311941a77ee91ee.jpg,0,6,0.000000,"[-0.10261335, -0.15464951, 0.022838585, 0.0013..."


In [ ]:
labels = kmeans.labels_
(labels == 0).sum()

228

In [ ]:
prob_table[labels == 0]['prob'].mean()

0.02745686099418714

In [ ]:
prob_table[labels == 1]['prob'].mean()

0.02015527904783615